In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def f(x):
    return 3*x**2 - 4*x + 5

In [ ]:
f(3.0)

In [ ]:
xs = np.arange(-5, 5, 0.25)
ys = f(xs)
plt.plot(xs, ys)

In [ ]:
h = 0.000000001
x = 3.0
(f(x+h) - f(x)) / h

In [ ]:
h = 0.0000001
x = -3.0
(f(x+h) - f(x)) / h

In [ ]:
a = 2.0
b = -3.0
c = 10.0
d = a*b + c
print(d)

In [ ]:
h = 0.0001

a = 2.0
b = -3.0
c = 10.0

d1 = a*b + c
d2 = (a+h)*b + c

print('d1', d1)
print('d2', d2)
print('slope', (d2-d1)/h)

In [ ]:
h = 0.0001

a = 2.0
b = -3.0
c = 10.0

d1 = a*b +c
d2 = a*(b+h) + c

print('d1', d1)
print('d2', d2)
print('slope', (d2-d1)/h)

In [ ]:
h = 0.0001

a = 2.0
b = -3.0
c = 10.0

d1 = a*b + c
d2 = a*b + c + h

print('d1', d1)
print('d2', d2)
print('slope', (d2-d1)/h)

In [ ]:
class Value:
    def __init__(self, data, label='', children=(), op=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self.label = label
        self.prev = set(children)
        self.op = op
    
    def __repr__(self):
        return f"Value(data={self.data})"
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, children=(self, other), op='+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad

        out._backward = _backward
        return out

    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, children=(self, other), op='*')

        def _backward():
            self.grad += out.grad * other.data
            other.grad += out.grad * self.data

        out._backward = _backward
        return out

    def __rmul__(self, other):
        return self * other

    def __radd__(self, other):
        return self + other

    def __rsub__(self, other):
        return other + (-self)

    def __truediv__(self, other):
        return self * (other ** -1)

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only int and float supported for power"
        out = Value(self.data ** other, children=(self,), op=f'**{other}')

        def backward():
            self.grad += out.grad * other * self.data ** (other - 1)

        out._backward = backward
        return out

    def tanh(self):
        t = math.tanh(self.data)
        out = Value(t, children=(self,), op='tanh')

        def _backward():
            self.grad += out.grad * (1 - t*t)

        out._backward = _backward
        return out

    def exp(self):
        e = math.exp(self.data)
        out = Value(e, children=(self,), op='exp')

        def _backward():
            self.grad += out.grad * e

        out._backward = _backward
        return out

    def backward(self):
        
        topo = []
        visited = set()
        def build_topo(v):
            if v in visited:
                return
            visited.add(v)
            for p in v.prev:
                build_topo(p)
            topo.append(v)
        
        build_topo(self)
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

In [ ]:
from graphviz import Digraph

def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v.prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'})

    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        dot.node(
            name = uid,
            label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad),
            shape = 'record'
            )
        if n.op:
            dot.node(
                name = uid + n.op,
                label = n.op
                )
            dot.edge(uid + n.op, uid)

    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2.op)

    return dot

In [ ]:
# inputs x1, x2
x1 = Value(2.0, 'x1')
x2 = Value(0.0, 'x2')
# weights w1, w2
w1 = Value(-3.0, 'w1')
w2 = Value(1.0, 'w2')
# bias b
b = Value(6.8813735870195432, 'b')
# x1*w1 + x2*w2 + b
x1w1 = x1 * w1; x1w1.label = 'x1w1'
x2w2 = x2 * w2; x2w2.label = 'x2w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1w1x2w2'
n = x1w1x2w2 + b; n.label = 'n'
o = n.tanh(); o.label = 'o'


In [ ]:

draw_dot(o)

In [ ]:
o.grad = 1.0
o.backward()
n.backward()
b.backward()
x1w1x2w2.backward()
x1w1.backward()
x2w2.backward()
x2.backward()
w2.backward()
x1.backward()
w1.backward()

In [ ]:
o.backward()

In [ ]:
a = Value(2.0, 'a')
b = a + a; b.label = 'b'
b.backward()
draw_dot(b)

In [ ]:
a = Value(-2.0, 'a')
b = Value(3.0, 'b')
d = a * b; d.label = 'd'
e = a + b; e.label = 'e'
f = d * e; f.label = 'f'

f.backward()

draw_dot(f)

In [ ]:
import torch

In [ ]:
torch.Tensor([[1,2,3],[4,5,6]]).double().dtype

In [ ]:
x1 = torch.Tensor([2.0]).double() ; x1.requires_grad = True
x2 = torch.Tensor([0.0]).double() ; x2.requires_grad = True
w1 = torch.Tensor([-3.0]).double() ; w1.requires_grad = True
w2 = torch.Tensor([1.0]).double() ; w2.requires_grad = True
b = torch.Tensor([6.8813735870195432]).double() ; b.requires_grad = True
n = x1*w1 + x2*w2 + b
o = torch.tanh(n)

print(o.data.item())
o.backward()

print('------')
print('x2', x2.grad.item())
print('x1', x1.grad.item())
print('w2', w2.grad.item())
print('w1', w1.grad.item())

In [ ]:
import random

class Neuron:
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1))
    
    def __call__(self, x):
        act = sum((wi*xi for wi,xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out

    def parameters(self):
        return self.w + [self.b]

class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs

    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]
    
class MLP:

    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [ ]:
n = MLP(3, [4, 4, 1])
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]
ys = [1.0, -1.0, -1.0, 1.0]

In [ ]:
for k in range(20):

    # forward pass
    ypred = [n(x) for x in xs]
    loss = sum(((ygt-yout)**2 for ygt, yout in zip(ys, ypred)))

    # backward pass
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()

    # update parameters
    for p in n.parameters():
        p.data -= 0.01 * p.grad

    print(k, loss.data)